In [22]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import itertools
import os
import ipywidgets as widgets

# change working directory to location of notebook/local starlog files
stardir = r'C:\Users\erinr\Desktop\starlogs'
os.chdir(stardir)

# list of starlog files to work with
starlogs = ['2021-0513-030617.starLog.sent.txt', '2021-0514-023207.starLog.sent.txt', '2021-0515-023347.starLog.sent.txt', '2021-0516-021727.starLog.sent.txt', '2021-0517-022130.starLog.sent.txt', '2021-0520-035716.starLog.sent.txt', 
            '2021-0524-021223.starLog.sent.txt', '2021-0527-075226.starLog.sent.txt', '2021-0528-035422.starLog.sent.txt', '2021-0529-020921.starLog.sent.txt', '2021-0530-022114.starLog.sent.txt']

# Working with local data (file list above: 2021-05-13 - 2021-05-30, skipping a few)

In [2]:
data = []

# Pull in starlog data, skipping the header and also skipping the fsnr lines. Does not
# work for a file which has multiple headers at the moment.

for starlog in starlogs:
    with open(starlog, 'r') as f:
        data.append(list(itertools.islice(f, 18, None, 2)))

# Combine all the starlogs into one list of entries, then split each entry
# which is a single string containing the entire entry into a list of the individual column entries
for i in range(len(data)):
    data[i] = [data[i][j].split() for j in range(len(data[i]))]

# Pull out only the coherent scans
data_clean = []
for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j][1] != 'I' and data[i][j][9] != '-1.000' and data[i][j][12] != '-1.000': # the and statements are in case a non-coherent entry was under a different label
            data_clean.append(data[i][j])

# Make arrays of the star names, hour angles, and offsets for each individual entry
stars = np.array([data_clean[i][2] for i in range(len(data_clean))])
angles = np.array([data_clean[i][4] for i in range(len(data_clean))]).astype(np.float)
b1 = np.array([data_clean[i][9] for i in range(len(data_clean))]).astype(np.float)
b4 = np.array([data_clean[i][12] for i in range(len(data_clean))]).astype(np.float)

# Dictionaries to contain the hour angles and offsets for each observation for each unique star
unique_stars = set(stars)
b1_dict = {star: [] for star in unique_stars}
b4_dict = {star: [] for star in unique_stars}
angles_dict = {star: [] for star in unique_stars}

# Sort the starlog entries into the dictionaries
for star in unique_stars:
    for i in range(len(stars)):
        if stars[i] == star:
            b1_dict[star].append(b1[i])
            b4_dict[star].append(b4[i])
            angles_dict[star].append(angles[i])

In [3]:
# Fit the baseline offsets for each baseline/star to a quadratic fit
# and make a dictionary of the coefficients

polydict = {star: [] for star in unique_stars}
for star in unique_stars:
    polydict[star].append(np.polyfit(angles_dict[star], b1_dict[star],2))
    polydict[star].append(np.polyfit(angles_dict[star], b4_dict[star],2))

In [4]:
# Function to plot the offsets and the polynomial fits for a given star, and to
# display calculated offsets for a given hour angle for that star

def plot_offsets(my_star, hour_angle=None):
    if my_star == None: # default value of the dropdown list is None when this code block is run
        pass
    else:        
        fig, ax = plt.subplots(1,1,figsize=(15,8))

        # Scatter plot the hour angles and offsets for the star
        ax.scatter(angles_dict[my_star], b1_dict[my_star], label='AC-AE, y=%.4fx^2 + %.4fx + %.4f' % (polydict[my_star][0][0], polydict[my_star][0][1], polydict[my_star][0][2]),c='lime')
        ax.scatter(angles_dict[my_star], b4_dict[my_star], label='E6-AE, y=%.4fx^2 + %.4fx + %.4f' % (polydict[my_star][1][0], polydict[my_star][1][1], polydict[my_star][1][2]),c='b')
        
        # Plot the polynomial fit for each baseline
        xs=np.linspace(np.amin(angles_dict[my_star]) - 0.5, np.amax(angles_dict[my_star]) + 0.5, 1000)
        ax.plot(xs, (polydict[my_star][0][0] * (xs**2)) + (polydict[my_star][0][1]* xs) + polydict[my_star][0][2],c='lime')
        ax.plot(xs, (polydict[my_star][1][0] * (xs**2)) + (polydict[my_star][1][1]* xs) + polydict[my_star][1][2],c='b')
        
        # Other plot setup
        ax.set_ylim([0,3])
        ax.set_title(my_star,fontsize=24)
        ax.set_ylabel('Baseline Offset (mm)',fontsize=20)
        ax.set_xlabel('Hour Angle', fontsize=20)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.legend(fontsize=16)
        
        plt.show()
    
        # Text box widgets to display the calculated baseline offsets for a given hour angle
        acae = widgets.Text('%.3f' % ((polydict[my_star][0][0] * (hour_angle**2)) + (polydict[my_star][0][1]* hour_angle) + polydict[my_star][0][2]),description='AC-AE Offset')
        e6ae = widgets.Text('%.3f' % ((polydict[my_star][1][0] * (hour_angle**2)) + (polydict[my_star][1][1]* hour_angle) + polydict[my_star][1][2]),description='E6-AE Offset')
        display(acae)
        display(e6ae)

# Widgets that interacts with the above function: dropdown list to select the star that the above function will plot,
# and text box to pass the hour angle that the function will take and use to calculate offsets
starbox = widgets.interact(plot_offsets,my_star=widgets.Dropdown(options=unique_stars, value=None, description='Star', disabled=False), hour_angle=widgets.FloatText(description='Hour Angle'))


interactive(children=(Dropdown(description='Star', options=('FKV0396', 'FKV0681', 'BSC5849', 'FKV0576', 'FKV07…

# Scrape from web

In [14]:
import requests
from bs4 import BeautifulSoup, SoupStrainer

In [15]:
# Connect to the starlog archive page and pull all of its html "content"
r = 'http://sextans.lowell.edu/npoi-internal/observing/starLogs/starLogArchive.html'
content = requests.get(r,auth=('npoiuser','H8NoFringes')).content

# Find and make a list of all the starlog links on the archive page
all_starlogs = []
for link in BeautifulSoup(content, parse_only=SoupStrainer('a')):
    if hasattr(link, "href"):
        all_starlogs.append(link['href'])

In [16]:
# Text dialogue to find an individual starlog: user provides month and year and then can
# select from a list of logs from that timeframe

year = input('From what year do you want to look at logs from? ')
year_str = '/' + year + '-'
year_logs = []
for log in all_starlogs:
    if year_str in log:
        year_logs.append(log)
month = input('Found ' + str(len(year_logs)) + ' starlogs from the year ' + year + '. What month would you like to look at? (Enter in MM format) ')
month_str = '/' + year + '-' + month
month_logs = []
for log in year_logs:
    if month_str in log:
        month_logs.append(log)
for i in range(len(month_logs)):
    print(str(i+1) + '. ' + month_logs[i])
final_file = int(input('Found ' + str(len(month_logs)) + ' starlogs from ' + year + '-' + month + ', seen above. Choose the number of the file you would like to look at.'))
test_link = 'http://sextans.lowell.edu/npoi-internal/observing/starLogs/' + month_logs[final_file-1]

print(test_link)

From what year do you want to look at logs from? 2021
Found 130 starlogs from the year 2021. What month would you like to look at? (Enter in MM format) 06
1. starLogDir/2021-0602-025028.starLog.sent
2. starLogDir/2021-0602-044619.starLog.sent
3. starLogDir/2021-0603-003136.starLog.sent
4. starLogDir/2021-0603-060005.starLog.sent
5. starLogDir/2021-0604-023612.starLog.sent
6. starLogDir/2021-0607-025339.starLog.sent
7. starLogDir/2021-0610-040321.starLog.sent
8. starLogDir/2021-0611-031045.starLog.sent
9. starLogDir/2021-0612-030712.starLog.sent
10. starLogDir/2021-0613-022952.starLog.sent
11. starLogDir/2021-0614-024428.starLog.sent
12. starLogDir/2021-0615-033540.starLog.sent
13. starLogDir/2021-0616-041056.starLog.sent
14. starLogDir/2021-0618-232959.starLog.sent
Found 14 starlogs from 2021-06, seen above. Choose the number of the file you would like to look at.11
http://sextans.lowell.edu/npoi-internal/observing/starLogs/starLogDir/2021-0614-024428.starLog.sent


In [17]:
# Get the data for the selected log
test_log = requests.get(test_link,auth=('npoiuser','H8NoFringes')).content
test_log_decode = test_log.decode() # decodes data from "byte" format to regular string format

In [18]:
# Split data into list of individual entries, skipping fsnr lines, and then split each individual entry into a list of column entries
data = test_log_decode.split('\n')[18::2]
data = [data[i].split() for i in range(len(data))]


# Pull out only coherent scan entries
data_clean = []

for i in range(len(data)):
    if data[i] != [] and data[i][1] != 'I' and data[i][9] != '-1.000' and data[i][12] != '-1.000':
        data_clean.append(data[i])

# Pull out star names, hour angles, and baseline offsets for each entry
stars = np.array([data_clean[i][2] for i in range(len(data_clean))])
angles = np.array([data_clean[i][4] for i in range(len(data_clean))]).astype(np.float)
b1 = np.array([data_clean[i][9] for i in range(len(data_clean))]).astype(np.float)
b4 = np.array([data_clean[i][12] for i in range(len(data_clean))]).astype(np.float)

# Dictionaries to contain the hour angles and offsets for each observation for each unique star
unique_stars = set(stars)
b1_dict = {star: [] for star in unique_stars}
b4_dict = {star: [] for star in unique_stars}
angles_dict = {star: [] for star in unique_stars}

# Sort the data into the dictionaries
for star in unique_stars:
    for i in range(len(stars)):
        if stars[i] == star:
            b1_dict[star].append(b1[i])
            b4_dict[star].append(b4[i])
            angles_dict[star].append(angles[i])

In [19]:
# Fit the baseline offsets for each baseline/star to a quadratic fit
# and make a dictionary of the coefficients

polydict = {star: [] for star in unique_stars}
for star in unique_stars:
    polydict[star].append(np.polyfit(angles_dict[star], b1_dict[star],2))
    polydict[star].append(np.polyfit(angles_dict[star], b4_dict[star],2))

C:\Users\erinr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\erinr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\erinr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\erinr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
# Function to plot the offsets and the polynomial fits for a given star, and to
# display calculated offsets for a given hour angle for that star

def plot_offsets(my_star, hour_angle=None):
    if my_star == None: # default value of the dropdown list is None when this code block is run
        pass
    else:        
        fig, ax = plt.subplots(1,1,figsize=(15,8))

        # Scatter plot the hour angles and offsets for the star
        ax.scatter(angles_dict[my_star], b1_dict[my_star], label='AC-AE, y=%.4fx^2 + %.4fx + %.4f' % (polydict[my_star][0][0], polydict[my_star][0][1], polydict[my_star][0][2]),c='lime')
        ax.scatter(angles_dict[my_star], b4_dict[my_star], label='E6-AE, y=%.4fx^2 + %.4fx + %.4f' % (polydict[my_star][1][0], polydict[my_star][1][1], polydict[my_star][1][2]),c='b')
        
        # Plot the polynomial fit for each baseline
        xs=np.linspace(np.amin(angles_dict[my_star]) - 0.5, np.amax(angles_dict[my_star]) + 0.5, 1000)
        ax.plot(xs, (polydict[my_star][0][0] * (xs**2)) + (polydict[my_star][0][1]* xs) + polydict[my_star][0][2],c='lime')
        ax.plot(xs, (polydict[my_star][1][0] * (xs**2)) + (polydict[my_star][1][1]* xs) + polydict[my_star][1][2],c='b')
        
        # Other plot setup
        ax.set_ylim([0,3])
        ax.set_title(my_star,fontsize=24)
        ax.set_ylabel('Baseline Offset (mm)',fontsize=20)
        ax.set_xlabel('Hour Angle', fontsize=20)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.legend(fontsize=16)
        
        plt.show()
    
        # Text box widgets to display the calculated baseline offsets for a given hour angle
        acae = widgets.Text('%.3f' % ((polydict[my_star][0][0] * (hour_angle**2)) + (polydict[my_star][0][1]* hour_angle) + polydict[my_star][0][2]),description='AC-AE Offset')
        e6ae = widgets.Text('%.3f' % ((polydict[my_star][1][0] * (hour_angle**2)) + (polydict[my_star][1][1]* hour_angle) + polydict[my_star][1][2]),description='E6-AE Offset')
        display(acae)
        display(e6ae)

# Widgets that interacts with the above function: dropdown list to select the star that the above function will plot,
# and text box to pass the hour angle that the function will take and use to calculate offsets
starbox = widgets.interact(plot_offsets,my_star=widgets.Dropdown(options=unique_stars, value=None, description='Star', disabled=False), hour_angle=widgets.FloatText(description='Hour Angle'))


interactive(children=(Dropdown(description='Star', options=('BSC5993', 'FKV0730', 'FKV0516', 'BSC6378', 'FKV07…